# 初始化，导入相关参数

In [1]:
from auto_ensemble import AutoModelEnsemble
import pandas as pd
from etide.model_evaluation import price_accuracy_smape
from etide.model.model_ensemble import ModelEnsemble
import warnings
warnings.filterwarnings("ignore")
from etide.utils import get_date
# 输入原始历史数据
df = pd.read_csv('data/shandong_0919.csv',index_col=[0], parse_dates=True)

# 子模型及标签列名
model_cols = ['模型1', '模型2', '模型3', '模型4']
target_col = '日前电价_hongtu110'

# 待预测起止时间
test_st = '2025-01-01'
test_et = '2025-09-19'

# 初始权重
# init_opt_weights = [0.1978, 0.3173, 0.4849,0.0]
init_opt_weights = [0.239, 0.3568,0.1705,0.2337]
# init_opt_weights = [0.25, 0.25,0.25,0.25]
# 超参更新频率
hype_params_renew_freq = [45]

# 超参寻优网格
params_grid = {'threshold':[0.01, 0.05, 0.1, 0.15],
              'decay':[0.9, 0.95],
              'window_len_range':[15,30,45, 60,75, 90,105, 120]} # 5~90

# 初始化AutoModelEnsemble，调用run函数进行回测，输出回测区间精度及最新权重

In [2]:
preds = []
for h in hype_params_renew_freq:
    ame = AutoModelEnsemble(model_cols,          # 子模型
                            target_col,          # 历史日前电价
                            params_grid,         # 超参寻优网格
                            init_opt_weights,    # 初始子模型权重
                            h,                   # 超参更新频率
                            price_accuracy_smape,# 评估函数
                            True                 # 评估函数是否越大越好
                           )
    
    pred, opt_weights = ame.run(df,     # 历史数据
                   test_st,# 预测起始日期
                   test_et # 预测结束日期
                  )
    preds.append(pred)

2025-10-16 09:09:03.328 | INFO     | auto_ensemble:run:244 - 2025-01-01更新超参, best_threshold, best_decay, best_window_len_range为(0.1, 0.95, 120)
2025-10-16 09:10:25.933 | INFO     | auto_ensemble:run:244 - 2025-02-15更新超参, best_threshold, best_decay, best_window_len_range为(0.1, 0.9, 60)
2025-10-16 09:11:52.512 | INFO     | auto_ensemble:run:244 - 2025-04-01更新超参, best_threshold, best_decay, best_window_len_range为(0.1, 0.9, 120)
2025-10-16 09:13:05.273 | INFO     | auto_ensemble:run:244 - 2025-05-16更新超参, best_threshold, best_decay, best_window_len_range为(0.01, 0.95, 15)
2025-10-16 09:14:16.333 | INFO     | auto_ensemble:run:244 - 2025-06-30更新超参, best_threshold, best_decay, best_window_len_range为(0.05, 0.9, 15)
2025-10-16 09:15:09.499 | INFO     | auto_ensemble:run:244 - 2025-08-14更新超参, best_threshold, best_decay, best_window_len_range为(0.05, 0.9, 90)
2025-10-16 09:15:10.840 | INFO     | auto_ensemble:run:270 - 回测完成,在45天的超参更新频率下,2025-01-01~2025-09-19的price_accuracy_smape为0.8722392041988778,

# 对于不同的超参更新频率，查看回测区间范围内任意区间的精度，为超参更新频率的确定提供参考

In [6]:
eval_st = '2025-01-01'
eval_et = '2025-08-26'
for i in range(len(hype_params_renew_freq)):
    print(f'每{hype_params_renew_freq[i]}天更新一次超参：{price_accuracy_smape(df.loc[eval_st:eval_et, target_col].values, preds[i].loc[eval_st:eval_et].values.flatten()):.4f}')

每45天更新一次超参：0.8703
